- **Author:** [Dace Apšvalka](https://www.mrc-cbu.cam.ac.uk/people/dace.apsvalka/) 
- **Date:** August 2024  
- **conda environment**: I used the [fMRI workshop's conda environment](https://github.com/MRC-CBU/COGNESTIC/blob/main/mri_environment.yml) to run this notebook and any accompanied scripts.

# Neuroimaging data manipulation

Adapted from:
- https://carpentries-incubator.github.io/SDC-BIDS-IntroMRI/anatomy-of-nifti.html and 
- https://github.com/miykael/workshop_pybrain

We'll be exploring two libraries: [NiBabel](http://nipy.org/nibabel/) and [Nilearn](https://nilearn.github.io/stable/index.html). 

**NiBabel** gives read and write access to common neuroimaging file formats. NiBabel’s API gives full or selective access to header information (metadata), and image data is made available via NumPy arrays.

"**Nilearn** enables **approachable and versatile analyses of brain volumes**. It provides statistical and machine-learning tools, with instructive documentation & open community. It supports general linear model (GLM) based analysis and leverages the scikit-learn Python toolbox for multivariate statistics with applications such as predictive modelling, classification, decoding, or connectivity analysis."

----------

**Table of contents**  
1. Setup   
2. Loading and inspecting images in `nibabel`  
2.1. Header   
2.2. Data  
2.3. Affine   
3. Image manipulation with `nilearn`   
3.1. The mean image](#toc3_1_)    
3.2. Resample image to a template   
3.3. Smooth an image 
3.4. Plotting a time course   
3.5. Masking an image


-------

## Setup

In [ ]:
import nibabel as nib

from nilearn import plotting
from nilearn import image as nli

import matplotlib.pyplot as plt
import pylab as plt

import numpy as np
## Set numpy to print 3 decimal points and suppress small values
np.set_printoptions(precision=3, suppress=True)

## Loading and inspecting images in `nibabel`

First, we will use the `load()` function to create a `NiBabel` image object from a NIfTI file. 

We’ll load in an example `T1w` and `BOLD` images that we will retrieve from our `BIDS` dataset.


In [ ]:
from bids.layout import BIDSLayout

fmri_data_dir = 'FaceProcessing/data'

# Initialize the BIDS layout
layout = BIDSLayout(fmri_data_dir)

# Get subject's T1w image and all Bold images
t1_file = layout.get(subject='04', extension='nii.gz', datatype='anat', return_type='filename')
bold_files = layout.get(subject='04', extension='nii.gz', suffix='bold', return_type='filename')

# Load the T1 image and the 1st Bold image
t1_img = nib.load(t1_file[0])
bold_img = nib.load(bold_files[0])

# Print the shape of both images
print(f"The shape of the T1 image: {t1_img.shape}")
print(f"The shape of the Bold image: {bold_img.shape}")

Loading in a NIfTI file with `NiBabel` gives us a special type of data object which encodes all the information in the file. Each bit of information is called an attribute in Python’s terminology. To see all of these attributes, type `t1_img.` followed by pressing `Tab`. There are three main attributes that we’ll discuss today:
* `Header`
* `Data`
* `Affine`

### Header
`Header` contains metadata about the image, such as image dimensions, data type, etc.

In [ ]:
t1_hdr = t1_img.header
print(t1_hdr)

`t1_hdr` is a Python **dictionary**. Dictionaries are containers that hold pairs of objects - **keys** and **values**. 
We can access the value stored by a given key by typing: `t1_hdr['<key_name>']`.

In [ ]:
t1_hdr['magic']

**==================================================================================================**

**EXCERCISE**

Extract `pixdim` value from the `BOLD image` header.

In [5]:
# write your code here


**==================================================================================================**

### Data

As you’ve seen above, the header contains valuable metadata about the MR data we’ve loaded. Now, we'll move on to loading the actual image data itself. This can be done using the `get_fdata()` method.

In [ ]:
# Get the T1 and Bold image data
t1_data = t1_img.get_fdata()
bold_data = bold_img.get_fdata()

# How does the T1 data look like
print(t1_data)

The data is a **multidimensional array** representing the image.

How can we check the number of dimensions in the t1_data array? You can view all the available attributes by typing `t1_data.` and then pressing `Tab`.

In [ ]:
# T1 number of dimensions
print(f"T1w image dimensions: {t1_data.ndim}")

**==================================================================================================**

**EXCERCISE**

What's the imensions of our BOLD image?

In [8]:
# write your code here


**==================================================================================================**

In [ ]:
# How large each dimension is
print(f"T1w image shape is {t1_data.shape}")
print(f"BOLD image shape is {bold_data.shape}")

The first 3 numbers given here represent the number of values along a respective dimension *(x,y,z)*. For the `BOLD` image this brain was scanned in `33` axial slices with a resolution of `64 x 64` voxels per slice. That means there are:

`64 * 64 * 33 = 135,168` voxels in total. And the BOLD signal was sampled `208` times. 

Let’s see the type of data inside of the array.


How do we examine **what value a particular voxel is**? We can inspect the value of a voxel by selecting an index as follows:

`data[x,y,z]`

So for example we can inspect a voxel at coordinates `(20,60,50)` by doing the following:

In [ ]:
# A value of a T1 image voxel at coordinates (20,60,50)
t1_data[19, 59, 49]

**NOTE**: Python uses **zero-based indexing**. The first item in the array is item `0`. The second item is item `1`, the third is item `2`, etc.

We can also extract data from a **slice** for visualisation and analysis. Slicing does exactly what it sounds like: from our 3D volume, we extract a 2D slice of the data. Below is an example of slicing from left to right (sagittal slicing, along the x-axis). In this case, we'll view the 20th slice.

In [ ]:
# Values of the T1 image's 20th sagittal slice
x_slice = t1_data[19, :, :]
print(x_slice)

This is similar to the indexing we did before to pull out a single voxel. However, instead of providing a value for each axis, the `:` indicates that we want to grab all values from that particular axis.

In [ ]:
z_slice = t1_data[:, :, 2]
print(z_slice)

We’ve been looking at voxel nummerical values, but we have no idea what the images actually look like! Let's look how the `100` slice of each of the `3` dimensions of T1 image look. 

In [ ]:
slices = [t1_data[99, :, :], t1_data[:, 99, :], t1_data[:, :, 99]]

fig, axes = plt.subplots(1, len(slices), figsize=(15,15))
for i, slice in enumerate(slices):
    axes[i].imshow(slice.T, cmap="gray", origin="lower")


### Affine

The final important piece of metadata associated with an image file is the **affine matrix**. The affine matrix defines the position of the image data in a reference space.

A voxel coordinate by itself tells us very little about where the data originates within the scanner. For example, if we have the voxel coordinate (26, 30, 16), without additional information, we wouldn’t know if this position is on the left or right side of the brain, or if it came from the left or right of the scanner.

This is because the scanner can collect voxel data in almost any arbitrary position and orientation within the magnet.

For instance, BOLD images are typically acquired at a different angle and with smaller coverage than T1-weighted anatomical images, resulting in different bounding boxes. 

<img align="centre" src="https://nipy.org/nibabel/_images/localizer.png" width="50%">

Additionally, the center of the BOLD image data is often not located at the exact center of the magnet bore (the magnet’s *isocenter*).

In our case, we have both an anatomical and a BOLD scan. Later, we will want to relate the data from the subject's _bold.nii.gz file to the same subject’s _T1w.nii.gz file. However, this is not straightforward, as the anatomical image and BOLD image were acquired with different orientations and fields of view, meaning the voxel coordinates in the BOLD image refer to different locations in the magnet compared to the anatomical image.

We solve this by using the affine matrix, which keeps track of the relationship between voxel coordinates and a reference space (e.g., the magnet space). The affine array stores how voxel coordinates in the image data correspond to coordinates in the reference space. Knowing this relationship for both images allows us to align the voxel coordinates of the BOLD image to the spatially equivalent coordinates in the T1-weighted image.

The origin of this reference system is at the magnet isocenter, at coordinate (0, 0, 0). The scanner’s axes, measured in mm, pass through this point. If the subject is lying face up, head first in the scanner, the axes are aligned with the subject’s head:

* The scanner's left/right axis corresponds to the subject's left-**right** axis. 
* The scanner's floor/ceiling axis corresponds to subject's posterior-**anterior** axis.
* The scanner's bore axid corresponds to the subject's inferior-**superior** axis..

This subject-centered scanner coordinate system is commonly used in neuroimaging and is referred to as **scanner RAS+** (right, anterior, superior). The **+** indicates that the right, anterior, and superior directions are positive on these axes (while left, posterior, and inferior are negative). **Note**: **Right** refers to the subject’s **right** side."

<img align="left" src="https://people.cas.sc.edu/rorden/anatomy/tspace.gif" width="30%">

<img align="right" src="https://www.slicer.org/w/img_auth.php/2/22/Coordinate_sytems.png" width="70%">

Below is the affine matrix for our anatomical `T1w` data. This matrix relates the **voxel coordinates** to the **world (scanner) coordinates** in **RAS+** space.

In [ ]:
t1_affine = t1_img.affine
print(t1_affine)

In the image header, the different `sform_code` and `qform_code` values specify which RAS+ space the sform affine refers to, with these interpretations:

| Code | Label     | Meaning                       |
|------|-----------|--------------------------------|
| 0    | unknown   | sform not defined              |
| 1    | scanner   | RAS+ in scanner coordinates    |
| 2    | aligned   | RAS+ aligned to some other scan|
| 3    | talairach | RAS+ in Talairach atlas space  |
| 4    | mni       | RAS+ in MNI atlas space        |


How 'shifted' is the T1 image's voxel space center from the reference space (scanner bore) center?

In [ ]:
# nibabel has a function apply_affine 
from nibabel.affines import apply_affine 

# the central voxel in the voxel space
t1_vox_center = (np.array(t1_data.shape) - 1) / 2.
print(f"The central voxel in the voxel space is {t1_vox_center.astype(int)}")

# distance from the reference space centre (in mm)
# voxel space's central voxel's location in the reference space
t1_vox_center_in_scanner = apply_affine(t1_img.affine, t1_vox_center)
print(f"The voxel space central voxel in the scanner space is at {t1_vox_center_in_scanner}")

That means the center of the T1 image field of view is **4.1 mm to the right** from the isocenter of the magnet, **18.8 mm anterior** to the isocenter and **1.2 mm above** (superior) the isocenter.

The parameters in the affine array can therefore give the position of any voxel coordinate, relative to the scanner RAS+ reference space.

When we register an image to a template, such as the **MNI template**, we obtain an affine matrix that defines the relationship between the voxels in the aligned image and the MNI RAS+ space. In the MNI reference space, the origin `(0, 0, 0)` is located at the **anterior commissure**.

## Image manipulation with `nilearn`

### The mean image

If you're using NiBabel to compute the mean image, you first need to load the image, extract the data, and then compute the mean.

With Nilearn, you can do all of this in a single line using the `mean_img function`.

In [ ]:
mean_img = nli.mean_img(bold_img)

In [ ]:
mean_data = mean_img.get_fdata()
mean_data.shape

Nilearn also offers interactive visualisation option, which is a great alternative to NiBabel's orthoview() function.

In [ ]:
plotting.view_img(mean_img, bg_img=mean_img)

### Resample image to a template

Using `resample_to_img`, we can resample one image to match the dimensions of another. For example, let's resample an anatomical `T1` image to the dimensions of the `mean` image we computed earlier.

In [ ]:
# image shapes before resampling
print([mean_img.shape, t1_img.shape])

In [ ]:
# resampling T1 to the mean Bold image
resampled_t1 = nli.resample_to_img(t1_img, mean_img)

# T1 image shape after resampling
resampled_t1.shape

How does the resampled `T1` image look like? Here we will use another `nilearn` plotting function that plots a static image. 

In [ ]:
plotting.plot_anat(t1_img, title = 'original t1', dim=-1)
plotting.plot_anat(resampled_t1, title = 'resampled t1', dim=-1)

### Smooth an image

Using `smooth_img`, we can quickly smooth any type of MRI image. For example, let's take the mean image from above and apply smoothing with different FWHM values.

In [ ]:
for fwhm in range(1, 12, 5):
    smoothed_img = nli.smooth_img(mean_img, fwhm)
    plotting.plot_epi(smoothed_img, title="Smoothing %imm" % fwhm,
                     display_mode='z', cmap='magma')

### Plotting a time course

Let's plot a time course of the central voxel in our BOLD imgage and some other random voxel.  

In [ ]:
# get the xyz of the center 
bold_vox_center = (np.array(bold_data.shape) - 1) / 2.
x, y, z, _ = bold_vox_center

# set the plot size
plt.figure(figsize=(12, 4))

# plot the central voxel time course
plt.plot(bold_data[int(x), int(y), int(z), :])

# plot some random voxel time course
plt.plot(bold_data[28, 45, 15, :])

# add legends to the plot
plt.legend(['center voxel', 'random voxel']);

Alternatively, we can use Nilearn's *NiftiSpheresMasker* function, which allows us to extract time series from a single voxel or a sphere around it. The input coordinates, in this case, must be in **world coordinates**.

In [ ]:
# Translate the two previously used voxel coordinates to the world coordinates
bold_center_coords = apply_affine(bold_img.affine, [x, y, z])
print(f"The center of the BOLD image in the world coordinates is {bold_center_coords}")

random_voxel_coords = apply_affine(bold_img.affine, [28, 45, 15])
print(f"The random voxel in the BOLD image in the world coordinates is {random_voxel_coords}")

In [25]:
# Extract the time series of the center and random voxels
from nilearn.maskers import NiftiSpheresMasker

coord_masker = NiftiSpheresMasker(
    [bold_center_coords, random_voxel_coords], t_r=2
)
coord_time_series = coord_masker.fit_transform(bold_img)

In [ ]:
# Plot the time series of the center and random voxels
plt.figure(figsize=(12, 4))
plt.plot(coord_time_series)
plt.legend(['center voxel', 'random voxel'])

### Masking an image

Let's take our BOLD functional image, compute its mean image, and apply a threshold to keep only the voxels with values higher than 95% of all voxels.

In [ ]:
#create the mean image
mean_img = nli.mean_img(bold_img)

#keep voxels that have a value that is higher than 95% of all voxels
thr = nli.threshold_img(mean_img, threshold='95%')

#let's see how the thresholded image look compared to the original mean image
plotting.view_img(thr, bg_img=mean_img)